# Pdf Q&A

In [17]:
# %pip install pypdf
# %pip install chromadb

In [18]:
from langchain.document_loaders import PyPDFLoader

### pdf loading

In [19]:
loader = PyPDFLoader("/Volumes/storage/patent/2023/20230102/1020150020108/B012/PDF/1020150020108.pdf")

In [20]:
document = loader.load()

In [21]:
document[0].page_content[:200]

'(19) 대한민국특허청(KR)\n(12) 등록특허공보(B1)(45) 공고일자   2023년01월02일\n(11) 등록번호   10-2483612\n(24) 등록일자   2022년12월28일\n(51) 국제특허분류(Int. Cl.)\n     H05K 1/18 (2006.01)  H05K 3/00 (2019.01)\n     H05K 3/46 (2006.01)\n(52)'

### Chunk 기반 텍스트 분할

In [22]:
from langchain.text_splitter import CharacterTextSplitter

In [23]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
texts = text_splitter.split_documents(document)

### embedding

In [24]:
import os
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
HUGGINGFACE_API_KEY = os.getenv('HUGGINGFACE_KEY')

In [25]:
# embedding 패키지 로딩
from langchain_community.embeddings import HuggingFaceHubEmbeddings

In [26]:
# embedding 클래스 정의 
EMBEDDING_BASE_URL=os.getenv("EMBEDDING_BASE_URL")
HUGGINGFACEHUB_API_TOKEN = os.getenv("HUGGUNGFACE_TOKEN")
embeddings = HuggingFaceHubEmbeddings(model=EMBEDDING_BASE_URL, huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN)


In [27]:
from langchain.vectorstores import Chroma

### 저장 및 검색

In [29]:
# Chroma DB 에 저장
docsearch = Chroma.from_documents(texts, embeddings)
# retriever 가져옴
retriever = docsearch.as_retriever()

### 프로젝트 템플릿

In [30]:
from langchain import hub
rag_prompt = hub.pull("rlm/rag-prompt")
rag_prompt

ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

### llm 생성

In [31]:
from langchain.chat_models import ChatOpenAI

In [32]:
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

### RAG Chain 생성

In [33]:
from langchain.schema.runnable import RunnablePassthrough

In [34]:
rag_chain = (
    {
        "context":retriever,
        "question": RunnablePassthrough()
    }
    |rag_prompt
    |llm    
)

In [35]:
rag_chain.invoke("이 문서의 명칭이 뭐지")

AIMessage(content='이 문서의 명칭은 "인쇄회로기판 및 그 제조방법"입니다.', response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 2343, 'total_tokens': 2364}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_bc2a86f5f5', 'finish_reason': 'stop', 'logprobs': None}, id='run-ee45ba4e-178f-4646-aa7a-5eb22948e2ea-0')

In [37]:
rag_chain.invoke("이 문서의 발명인, 출원인, 권리자를 추출해서 python 사전 구조로 표시해줘")

AIMessage(content='```python\n{\n    "발명자": ["이준성", "조석현", "이용삼"],\n    "출원인": "삼성전기주식회사",\n    "권리자": "삼성전기주식회사"\n}\n```', response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 1590, 'total_tokens': 1647}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_4e2b2da518', 'finish_reason': 'stop', 'logprobs': None}, id='run-bffc96fa-d3a8-49ab-82f2-2710676e11df-0')

In [38]:
rag_chain.invoke("입력된 문서의 수량은 몇개나 되지?")

AIMessage(content='입력된 문서의 수량은 총 4개입니다.', response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 2348, 'total_tokens': 2363}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_4e2b2da518', 'finish_reason': 'stop', 'logprobs': None}, id='run-791fdad4-8e01-413f-863c-8dd382ade62e-0')

In [39]:
rag_chain.invoke("입력된 모든 문서의 제목을 표시해줘 ")

AIMessage(content='"인쇄회로기판 및 그 제조방법"', response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 1575, 'total_tokens': 1587}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_4e2b2da518', 'finish_reason': 'stop', 'logprobs': None}, id='run-8e2c7297-ac48-4517-b9c6-0e94bfaa3aef-0')

In [40]:
rag_chain.invoke("입력된 문서의 발명의 배경 및 해결하고자 하는 문제, 문제 해결 방법을 python 사전 구조로 표시해줘 ")

AIMessage(content='```python\n{\n    "발명의 배경": "전자제품이 고집적화되고 얇아지면서 기판 위에 실장되는 소자들이 기판의 내부에 매립되는 임베디드 기판 기술이 발전하고 있다. 이러한 기술 발전으로 캐비티의 수와 크기가 증가하고, 기판의 휨 현상이 발생한다.",\n    "해결하고자 하는 문제": "기판의 휨 현상을 개선하여 임베디드 기판의 신뢰성을 향상시키는 것.",\n    "문제 해결 방법": "글라스 기판을 가공하여 더미칩을 완성하고, 이를 기판에 임베딩하여 휨 현상을 개선하는 인쇄회로기판을 제공한다."\n}\n```', response_metadata={'token_usage': {'completion_tokens': 170, 'prompt_tokens': 4788, 'total_tokens': 4958}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_4e2b2da518', 'finish_reason': 'stop', 'logprobs': None}, id='run-e1e07f5b-c11d-40fb-8cdb-a0aa7afc031d-0')

In [42]:
rag_chain.invoke("""
입력된 문서의 
발명의 배경 및 
해결하고자 하는 문제, 
문제 해결 방법을 추출하고, 
각각에 대해 한문장으로 요약 추출하고
python 사전 구조로 표시해줘""")

AIMessage(content='```python\n{\n    "발명의 배경": "전자제품의 고집적화와 얇아짐에 따라 기판 내부에 소자를 매립하는 임베디드 기판 기술이 발전하고 있으며, 캐비티의 수와 크기가 증가하면서 기판의 휨 현상이 문제로 대두되고 있다.",\n    "해결하고자 하는 문제": "글라스 기판을 직접 가공하여 더미칩을 완성하고 이를 기판에 임베딩하여 기판의 휨 현상을 개선함으로써 임베디드 기판의 신뢰성을 향상시키는 것이다.",\n    "문제 해결 방법": "캐비티를 갖는 코어 기판과 그 캐비티에 수용된 더미칩을 포함하는 인쇄회로기판을 제공하며, 더미칩을 준비하고 이를 수용할 캐비티를 갖는 코어 기판을 준비한 후 캐비티에 더미칩을 수용하는 방법을 제안한다."\n}\n```', response_metadata={'token_usage': {'completion_tokens': 225, 'prompt_tokens': 2148, 'total_tokens': 2373}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_4e2b2da518', 'finish_reason': 'stop', 'logprobs': None}, id='run-a965855f-d9ef-46f9-adee-45f04ed05efd-0')